This Notebook is a brief sketch of how to use the Deutsch-Jozsa algorithm.
We start by declaring all necessary imports.

In [1]:
from collections import defaultdict
from pyquil.api import SyncConnection
import numpy as np
from mock import patch

from grove.deutsch_jozsa.deutsch_jozsa import DeutschJosza

The Deutsch-Jozsa Algorithm can be used to determine if a binary-valued function is constant or balanced (returns 0 on exactly half of its inputs).

In [ ]:
constant_bitmap = {"00": "0",
                   "01": "0",
                   "10": "0",
                   "11": "0"}

To use the Deutsch-Jozsa algorithm on a Quantum Hardware we need to define the connection to the QVM or QPU. However we don't have a real connection in this notebook, so we just mock out the response. If you run this notebook, ensure to replace `cxn` with a pyQuil connection object.

In [4]:
with patch("pyquil.api.SyncConnection") as cxn:
    # Need to mock multiple returns as an iterable
    cxn.run_and_measure.side_effect = [
        (np.asarray([0, 0]))]

Now let's run Simon's algorithm. We instantiate the Simon object and then call its `find_mask` mehtod with the connection object and the 2-to-1 function whose mask we wish to find.
Finally we assert its correctness by checking with the mask we used to generate the function

In [5]:
dj = DeutschJosza()
is_constant = dj.is_constant(cxn, constant_bitmap)
assert is_constant, "The algorithm said the function was balanced."

If the assertion succeeded we know the algorithm returned the zero bitstring and successfully recognized the function as constant.